In [34]:
import nest_asyncio
import os

from langchain_anthropic import ChatAnthropic
from llama_index.core import PropertyGraphIndex 
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.indices.property_graph import VectorContextRetriever, CypherTemplateRetriever, LLMSynonymRetriever
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.llms.anthropic import Anthropic
from typing import List, Optional, Dict, Any, Set

In [2]:
nest_asyncio.apply()

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
CLAUDE_API_KEY = os.getenv('CLAUDE_API_KEY')
LLAMA_API_KEY = os.getenv('LLAMA_API_KEY')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["ANTHROPIC_API_KEY"] = CLAUDE_API_KEY
os.environ["LLAMA_CLOUD_API_KEY"] = LLAMA_API_KEY

In [4]:
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    max_tokens=4096,
    temperature=0.0,
    stop=["\n\nHuman"],
)

llama_llm = Anthropic(
    model="claude-3-5-sonnet-20240620",
    max_tokens=4096,
    temperature=0.0
)

llama_openai_embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [106]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "15082001"
NEO4J_DATABASE = "neo4j"

graph_store = Neo4jPropertyGraphStore(
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
    url=NEO4J_URI,
    refresh_schema=False,
)

In [107]:
index = PropertyGraphIndex.from_existing(
    llm = llama_llm,
    embed_model=llama_openai_embed_model,
    property_graph_store=graph_store,
)

### Vector Similarity Sub-Retriever

In [108]:
vector_retriever = VectorContextRetriever(
  index.property_graph_store,
  vector_store=index.vector_store,
  embed_model=llama_openai_embed_model,
)

### Entity Sub-Retriever

In [109]:
keyword_retriever = LLMSynonymRetriever(
    index.property_graph_store, 
    llm=llama_llm,
    path_depth=1,
)

In [69]:
retriever = index.as_retriever(sub_retrievers=[vector_retriever, keyword_retriever])

In [110]:
nodes = retriever.retrieve("What are some diabetes medications?")

In [223]:
for node in nodes:
    print("====" * 20)
    print(node.text)

diabetes medications 

  sulphonylureas 

 help the pancreas to release more insulin and the body to use glucose more effectively . 

 - chlorpropamide ( diabinese ® ) 
 - glibenclamide ( daonil ® ) 
 - gliclazide ( diamicron ® ) 
 - tolbutamide ( rastinon ® ) 
 - glimepiride ( amaryl ® ) 

 take the tablet(s ) daily immediately after meals ( except gliclazide which is consumed half an hour before meals ) . do not miss or delay meals . otherwise , your blood sugar level may fall too low . 

 sometimes other tablets ( e.g. those for blood pressure ) can react with anti - diabetes tablets . if in doubt , talk to your doctor or pharmacist . 

 alcohol interferes with some of these tablets : consult your pharmacist if you are planning to drink alcohol . for example , having alcohol with chlorpropamide causes side effects like redness and vomiting . 

  biguanides 

 help the body to use glucose more effectively . 

 - e.g. metformin ( glucophage ® ) 

 take the tablet(s ) with meals or aft